In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.stats import norm

import torch
device = "cuda"
torch.cuda.get_device_name(torch.cuda.current_device())

'GeForce RTX 3080'

In [3]:
from sklearn.datasets import load_iris
np.random.seed(0)

iris = load_iris()
X = iris['data']
Y = iris['target']
DATA = np.concatenate((X, np.reshape(Y, (len(Y), 1))), axis=1)
np.random.shuffle(DATA)
training_portion = 0.6

index = int(len(DATA) * training_portion)
TRAIN = DATA[:index, :]
TEST  = DATA[index:, :]

Xtrain, Ytrain = TRAIN[:, :-1], TRAIN[:, -1]
Xtest , Ytest  = TEST[:, :-1], TEST[:, -1]

In [4]:
Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

((90, 4), (90,), (60, 4), (60,))

In [6]:
from PCA_pvalue import PCA_pvalue
from Distance_Classifier import Distance_classifier

# m=PCA_pvalue(Xtrain, np.array(Ytrain, dtype=np.int32))
# m.fit()
# m.all_p_values(Xtest)

In [33]:
unit_projection(t)
t, t.grad

(tensor([0.5774, 0.5774, 0.5774], requires_grad=True),
 tensor([0.6667, 0.6667, 0.6667]))

In [8]:
def unit_projection(t):
    with torch.no_grad():
        t.set_(t / torch.norm(t))

In [9]:
def to_numpy(x):
    """Convert a PyTorch tensor to NumPy."""
    return x.squeeze().detach().cpu().numpy()


def to_torch(x, device='cpu'):
    return torch.tensor(x).unsqueeze(-1).to(device).float()

In [42]:
def onehot(labels, uniques=15):
    res=torch.zeros((labels.shape[0], uniques), dtype=torch.float64).to(device)
    for i in range(res.shape[0]):
        res[i,labels[i]] = 1
    return res

In [192]:
torch.tensor([[1,2,3],[4,5,6]])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [202]:
torch.matmul(torch.tensor([[3,2],[0.5,0.5]], dtype=torch.float), (torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float)))

tensor([[11.00, 16.00, 21.00],
        [ 2.50,  3.50,  4.50]])

In [ ]:
torch.tensor([
    []
])

In [119]:
torch.transpose(torch.tensor([[1,2,3],[4,5,6]]), 0, 1)*torch.tensor([[3,2],[1,1],[]])

tensor([[ 3,  8],
        [ 6, 10],
        [ 9, 12]])

In [11]:
def predictpvalue(phat, weight):
    return torch.matmul(weight, phat)

phat=torch.tensor([[1,2,3],[4,5,6]])
weight=torch.tensor([[3,2]])
print(predictpvalue(phat, weight).float())

torch.nn.MSELoss(reduction='sum')(predictpvalue(phat, weight).float(), torch.tensor([11,16,21]).float())

tensor([[11., 16., 21.]])


tensor(0.)

In [135]:
torch.set_printoptions(precision=2)

In [70]:
class Weighted_avg_ensemble:

    def __init__(self, classifiers=[Distance_classifier, PCA_pvalue], device='cpu'):
        # init classifiers
        self.classifiers = classifiers
        # init parameters
        self.device = device

    # @staticmethod
    # def unit_projection(t):
    #     # return None
    #     with torch.no_grad():
    #         t.set_(t / torch.norm(t))

    def fit_submodels(self, Xtrain: np.array, Ytrain: np.array):
        # record meta data
        self.classes = np.array(np.unique(Ytrain), dtype=np.long)

        # fit all submodels in the ensemble
        Ytrain = np.array(Ytrain, dtype=np.long)
        self.models = []
        for Classifier in self.classifiers:
            model = Classifier(Xtrain, Ytrain)
            model.fit()
            self.models.append(model)

        # init parameters
        self.parameters = torch.rand((len(self.classifiers), len(self.classes)),
                                     dtype=torch.float64, requires_grad=True, device=self.device)

    def forward(self, X: np.array):
        predictions = []
        for classifier in self.models:
            predictions.append(classifier.all_p_values(X))
        P = torch.tensor(predictions).to(self.device).permute(1,0,2)
        r = torch.sum(torch.mul(P, self.parameters**2), axis=1)
        phat = torch.mul(r, 1/torch.sum(self.parameters**2, axis=0))
        return phat

    def all_p_values(self, X):
        with torch.no_grad():
            return self.forward(X)

    def acc(self, Xtest: np.array, Ytest: np.array):
        Ytest = torch.tensor(Ytest, dtype=torch.float64, device=self.device)

        correct = torch.sum(Ytest==torch.argmax(self.all_p_values(Xtest), axis=1))
        total   = Ytest.shape[0]
        return correct / total

In [73]:
# model definition
model=Weighted_avg_ensemble(device='cuda')
model.fit_submodels(Xtrain, Ytrain)

# convert labels to onehot encoding
Ytrain = np.array(Ytrain, dtype=np.long)
Ytrain_onehot = onehot(Ytrain, np.unique(Ytrain).shape[0]).to(device)

# Hyperparam
batch_size = 10
optimizer  = torch.optim.Adam([model.parameters], lr=1e-2)
lossfn     = torch.nn.functional.mse_loss
epochs     = 20

# training loop
# training_iteration = (Xtrain.shape[0] // batch_size + 1) * epochs
for i in range(epochs):
    for j in range(batch_size):

        # get batch
        x = Xtrain[j:min(j+10,len(Xtrain))]
        y = Ytrain[j:min(j+10,len(Xtrain))]

        # forward pass and backward pass
        optimizer.zero_grad()
        yhat = model.forward(Xtrain)
        loss = lossfn(Ytrain_onehot, yhat)
        loss.backward()
        optimizer.step()

        print(f"iteration:{i*len(Xtrain)+j*batch_size} loss={loss.item()} acc={model.acc(Xtest, Ytest)}")

print("After training P-hat")
print(model.all_p_values(Xtest))
print("True Label")
print(Ytrain)
print(f"train ACC={model.acc(Xtrain, Ytrain)}")
print(f"test  ACC={model.acc(Xtest, Ytest)}")

iteration:0 loss=0.15753015339097498 acc=0.98333340883255
iteration:10 loss=0.1566190916004199 acc=0.98333340883255
iteration:20 loss=0.1556610359774231 acc=0.98333340883255
iteration:30 loss=0.15462606269035878 acc=0.98333340883255
iteration:40 loss=0.15369982115595332 acc=0.98333340883255
iteration:50 loss=0.1528504485122218 acc=0.98333340883255
iteration:60 loss=0.15203094828649402 acc=0.98333340883255
iteration:70 loss=0.1512148623322691 acc=0.98333340883255
iteration:80 loss=0.15039476141225988 acc=0.98333340883255
iteration:90 loss=0.1495732640448336 acc=0.98333340883255
iteration:90 loss=0.1487553373102846 acc=0.98333340883255
iteration:100 loss=0.1479443406334794 acc=0.98333340883255
iteration:110 loss=0.14714099491201316 acc=0.98333340883255
iteration:120 loss=0.14634379931364708 acc=0.98333340883255
iteration:130 loss=0.1455498940361815 acc=0.98333340883255
iteration:140 loss=0.1447559053215806 acc=0.98333340883255
iteration:150 loss=0.14395861158009032 acc=0.98333340883255
i

In [74]:
l=to_numpy(model.all_p_values(Xtest)).tolist()
for r in l:
    for i in r:
        print(f"{i:.2f}", end="\t")
    print()

0.00	0.00	0.58	
0.00	0.00	0.00	
0.00	0.00	0.15	
0.95	0.00	0.00	
0.00	0.03	0.66	
0.00	0.00	0.87	
0.32	0.00	0.00	
0.95	0.00	0.00	
0.00	0.00	0.47	
0.95	0.00	0.00	
0.95	0.00	0.00	
0.21	0.00	0.00	
0.00	0.79	0.01	
0.00	0.00	0.76	
0.00	0.00	0.17	
0.95	0.00	0.00	
0.95	0.00	0.00	
0.95	0.00	0.00	
0.00	0.44	0.03	
0.00	0.39	0.05	
0.26	0.00	0.00	
0.77	0.00	0.00	
0.00	0.79	0.04	
0.39	0.00	0.00	
0.00	0.00	0.01	
0.00	0.94	0.00	
0.00	0.00	0.01	
0.00	0.39	0.02	
0.17	0.00	0.00	
0.00	0.12	0.92	
0.06	0.00	0.00	
0.00	0.00	0.10	
0.95	0.00	0.00	
0.95	0.00	0.00	
0.00	0.00	0.66	
0.95	0.00	0.00	
0.00	0.00	0.32	
0.00	0.94	0.01	
0.00	0.94	0.00	
0.00	0.14	0.00	
0.00	0.00	0.41	
0.00	0.00	0.71	
0.00	0.14	0.44	
0.00	0.39	0.34	
0.66	0.00	0.00	
0.00	0.94	0.00	
0.00	0.00	0.71	
0.00	0.27	0.54	
0.99	0.00	0.00	
0.00	0.64	0.09	
0.00	0.87	0.00	
0.00	0.14	0.58	
0.00	0.56	0.05	
0.32	0.00	0.00	
0.56	0.00	0.00	
0.87	0.00	0.00	
0.00	0.01	0.71	
0.00	0.64	0.00	
0.00	0.00	0.02	
0.95	0.00	0.00	


In [61]:
print("After training P-hat")
print(torch.argmax(model.all_p_values(Xtest), axis=1))
print("True Label")
print(Ytest)
print(f"test  ACC={model.acc(Xtest, Ytest)}")

After training P-hat
tensor([2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 2, 1, 0, 1, 2, 2,
        0, 1, 1, 2, 1, 0, 0, 0, 2, 1, 2, 0], device='cuda:0')
True Label
[2. 0. 2. 0. 2. 2. 0. 0. 2. 0. 0. 0. 1. 2. 2. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 2. 1. 2. 1. 0. 2. 0. 2. 0. 0. 2. 0. 2. 1. 1. 1. 2. 2. 1. 1. 0. 1. 2. 2.
 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 2. 0.]
test  ACC=0.9666666984558105


In [ ]:
plt.scatter()

In [75]:
model.parameters

tensor([[ 6.6422e-01,  1.6387e-01,  8.3616e-01],
        [ 7.2779e-06, -1.3576e-07, -1.6002e-04]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)